In [ ]:
import os
from PIL import Image
import tensorflow as tf
import numpy as np
import cv2

import torchvision.transforms as transforms
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
import torch
import torch.nn as nn
from torch import Tensor

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        #dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()

        # Normalization Layer
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.LeakyReLU(0.1)
        #self.relu=nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # downsampling이 필요한 경우 downsample layer를 block에 인자로 넣어주어야함
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity  # residual connection
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(
        self,
        block: BasicBlock,
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer  # batch norm layer

        self.inplanes = 64  # input shape
        #self.dilation = 1  # dilation fixed
        self.groups = 1  # groups fixed

        # input block
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        #self.relu =nn.LeakyReLU(0.1)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # residual blocks
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)


        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: BasicBlock, planes: int, blocks: int,
                    stride: int = 1) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None

        # downsampling 필요할경우 downsample layer 생성
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes, stride),
                norm_layer(planes),
            )

        layers = []
        #layers.append(block(self.inplanes, planes, stride, downsample, self.groups,self.dilation, norm_layer))
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            norm_layer))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        print("forward")
        #print('input shape:', x.shape)
        x = self.conv1(x)
        #print('conv1 shape:', x.shape)
        x = self.bn1(x)
        #print('bn1 shape:', x.shape)
        x = self.relu(x)
        #print('relu shape:', x.shape)
        x = self.maxpool(x)
        #print('maxpool shape:', x.shape)

        x = self.layer1(x)
        #print('layer1 shape:', x.shape)
        x = self.layer2(x)
        #print('layer2 shape:', x.shape)
        x = self.layer3(x)
        #print('layer3 shape:', x.shape)
        x = self.layer4(x)
        #print('layer4 shape:', x.shape)

        x = self.avgpool(x)
        #print('avgpool shape:', x.shape)
        x = torch.flatten(x, 1)
        #print('flatten shape:', x.shape)
        x = self.fc(x)
        #print('fc shape:', x.shape)

        return x

# ResNet-50 모델 정의
model = ResNet(BasicBlock,[3,4,4,3],num_classes=5,zero_init_residual=True)

# 미리 학습된 가중치 불러오기
checkpoint = torch.load('/content/drive/MyDrive/model_checkpoint_epoch1.pt')

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()



def classify_faces(face_images):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    predictions = []
    '''
    for face in face_images:
        input_tensor = transform(face).unsqueeze(0)  # 배치 차원 추가
        with torch.no_grad():
            output = ResNet(input_tensor)
        _, predicted_class = torch.max(output, 1)
        predictions.append(predicted_class.item())
    '''

    input_tensor = transform(face_image).unsqueeze(0)  # Adding batch dimension
    with torch.no_grad():
        output = model(input_tensor)
    return output

from flask import Flask, request, jsonify
import io
from PIL import Image

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/detect_and_classify', methods=['POST'])
def detect_and_classify():
    # 클라이언트로부터 이미지 파일 받기
    image_file = request.files['image']
    image_bytes = image_file.read()
    image = Image.open(io.BytesIO(image_bytes))

    # 얼굴 분류
    predictions = classify_faces(image)

    # Softmax 함수를 사용하여 확률값으로 변환
    predictions = torch.softmax(predictions, dim=1)
    predictions = predictions.squeeze().tolist()
    #predictions= ["{:.3f}".format(value) for value in predictions]
    # 결과 반환
    #return jsonify(predictions.squeeze().tolist())
    return jsonify(predictions)
if __name__ == '__main__':
    app.run(debug=True)